In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
% pip install -r /content/drive/MyDrive/21FALL/EECS595/code/requirements.txt

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import json
with open("/content/drive/MyDrive/21FALL/EECS595/code/config.json", "r") as f:
  args = json.load(f)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
dev_data_df = pd.read_csv(args['DEV_PATH'], sep='\t', index_col=0)
print(dev_data_df.shape)
dev_data_df.head(5)

(500, 11)


,Resolved pattern,Article title,Section header,Previous context,Sentence,Follow-up context,Filler1,Filler2,Filler3,Filler4,Filler5
Id,,,,,,,,,,,
0,IMPLICIT REFERENCE,How to Detect Cleft Lip Early in Pregnancy,Treating Cleft Lip,1. Care for a newborn with a cleft lip. (....,2. Get surgery for ______ at 3 to 6 months old.,A pediatric surgeon will move the skin and mus...,the split,your chin,cleft lips,your baby,the ear
1,METONYMIC REFERENCE,How to Deal With Being Single and Feeling Lonely,Focusing on Yourself,(...) 3. Go on a trip. (...) 4. Get active an...,"If you are lonely, take up a new sport or ____...","This is great for your body, as well as your s...",freedom,circle,field,activity,form
2,FUSED HEAD,How to Cope With Anxiety and Depression,Seeking Professional Help,(...) 4. Allow medication time to work. If yo...,Most ______ take four to eight weeks to show a...,NaN,disorders,men,medications,people,procedures
3,FUSED HEAD,How to Be an Otaku (Anime Geek),Steps,1. Pick and watch some anime of your interest...,"If you want to have the edge over others, then...",2. Try watchingthe anime in the original Japan...,families,humans,men,people,females
4,METONYMIC REFERENCE,How to Uninstall Netflix on Samsung Smart TV,Steps,1. Press the Home button.,The home button is the button that resembles t...,This will open the Smart Hub.,video,interior,outline,movement,construction


In [ ]:
train_data_df = pd.read_csv(args['TRAIN_PATH'], sep='\t', index_col=0)
print(train_data_df.shape)
train_data_df.head(5)

(3991, 11)


,Resolved pattern,Article title,Section header,Previous context,Sentence,Follow-up context,Filler1,Filler2,Filler3,Filler4,Filler5
Id,,,,,,,,,,,
1,IMPLICIT REFERENCE,How to Talk to Your Partner About Their Weigh,Things You'll Need,(...) * Lose Weight the Healthy Way * Us...,* Lose Weight Fast with ______,,stress,plenty,strength,ease,exercise
2,ADDED COMPOUND,How to Change the Name of a Wireless Network,Steps,(...) 7. Enter a new name for your wireless n...,Save your new ______ name.,"Click on ""Apply,"" ""Save Settings,"" ""Save,"" or ...",program,host,web,connection,network
3,ADDED COMPOUND,How to Get Rid of Peeling Skin,Following a Basic Routine,(...) 6. Never tear away loose skin. (...) 7....,Exposure to direct sunlight can weaken your sk...,This is true regardless of whether your skin i...,exposure,sunlight,healthcare,heat,peeling
4,METONYMIC REFERENCE,How to Prevent HIV Infection,Sources and Citations.,HIV (Human Immunodeficiency Virus) and AIDS (...,HIV can infect anybody who has been exposed to...,The virus does not discriminate against who it...,levels,sources,areas,organisms,parts
5,ADDED COMPOUND,How to Make Sure Your Children Get Enough Vita...,Steps,(...) 3. Encourage outside play time. (...) 4...,Children lacking in Vitamin D can make up for ...,"Make sure to purchase ones that write ""Calcium...",multivitamin,zinc,glucose,nutrition,sunlight


In [ ]:
pd.read_csv(args['TRAIN_LABEL_PATH'], sep='\t', header=None)

,0,1
0,1_1,PLAUSIBLE
1,1_2,NEUTRAL
2,1_3,IMPLAUSIBLE
3,1_4,NEUTRAL
4,1_5,NEUTRAL
...,...,...
19970,3995_1,IMPLAUSIBLE
19971,3995_2,PLAUSIBLE
19972,3995_3,IMPLAUSIBLE
19973,3995_4,NEUTRAL


In [ ]:
train_label_df = pd.read_csv(args['TRAIN_LABEL_PATH'], sep='\t', header=None)
train_label_df.columns = ["Id_Filler", "Plausibility"]
train_label_df["Id"] = train_label_df["Id_Filler"].apply(lambda x: int(x.split("_")[0]))
train_label_df["Filler"] = train_label_df["Id_Filler"].apply(lambda x: int(x.split("_")[1]))
train_label_df.drop("Id_Filler", axis=1, inplace=True)
train_label_df.set_index(keys=["Id", "Filler"], inplace=True)
print(train_label_df.shape)
train_label_df.head(5)

(19975, 1)


Plausibility
Id Filler             
1  1         PLAUSIBLE
   2           NEUTRAL
   3       IMPLAUSIBLE
   4           NEUTRAL
   5           NEUTRAL

In [ ]:
train_score_df = pd.read_csv(args['TRAIN_SCORE_PATH'], sep='\t', header=None)
train_score_df.columns = ["Id_Filler", "Score"]
train_score_df["Id"] = train_score_df["Id_Filler"].apply(lambda x: int(x.split("_")[0]))
train_score_df["Filler"] = train_score_df["Id_Filler"].apply(lambda x: int(x.split("_")[1]))
train_score_df.drop("Id_Filler", axis=1, inplace=True)
train_score_df.set_index(keys=["Id", "Filler"], inplace=True)
print(train_score_df.shape)
train_score_df.head(5)

(19975, 1)


Score
Id Filler          
1  1       4.000000
   2       3.500000
   3       2.250000
   4       2.666667
   5       3.333333

In [ ]:
train_y_df = train_score_df.merge(train_label_df, how="inner", on=["Id", "Filler"])
train_y_df.head(5)

Score Plausibility
Id Filler                       
1  1       4.000000    PLAUSIBLE
   2       3.500000      NEUTRAL
   3       2.250000  IMPLAUSIBLE
   4       2.666667      NEUTRAL
   5       3.333333      NEUTRAL

In [ ]:
idx = pd.IndexSlice
train_y_df.loc[idx[[4],:], :]

Score Plausibility
Id Filler                    
4  1         4.5    PLAUSIBLE
   2         3.5      NEUTRAL
   3         2.0  IMPLAUSIBLE
   4         5.0    PLAUSIBLE
   5         2.5  IMPLAUSIBLE

In [ ]:
train_data_df.columns

Index(['Resolved pattern', 'Article title', 'Section header',
       'Previous context', 'Sentence', 'Follow-up context', 'Filler1',
       'Filler2', 'Filler3', 'Filler4', 'Filler5'],
      dtype='object')

In [ ]:
train_data_df["Resolved pattern"].value_counts()

ADDED COMPOUND         1000
METONYMIC REFERENCE     999
FUSED HEAD              997
IMPLICIT REFERENCE      995
Name: Resolved pattern, dtype: int64

In [ ]:
samples = [40, 2013, 259, 593, 1930, 1402, 3068]
for i in samples:
  print(f"Sample #{i + 1}:")
  print(train_data_df.loc[i].values)
  print(f"Label #{i + 1}")
  print(train_y_df.loc[idx[i,:], :])
  print("\n")

Sample #41:
['METONYMIC REFERENCE' 'How to Give Yourself a Caviar Manicure'
 'Stepping it Up'
 ' 1. Make it stand out by turning it into an accent nail. (...) 2. Mix it up with a multicolored caviar manicure. '
 'Instead of using just one color of microbeads on your nails, consider using a ______ of several different colors. '
 'Mix these microbeads up in their own bottle first, then apply them to your nails following the method above.'
 'makeup' 'color' 'mix' 'variety' 'nail']
Label #41
              Score Plausibility
Id Filler                       
40 1       1.666667  IMPLAUSIBLE
   2       1.000000  IMPLAUSIBLE
   3       5.000000    PLAUSIBLE
   4       3.500000      NEUTRAL
   5       3.500000      NEUTRAL


Sample #2014:
['METONYMIC REFERENCE' 'How to Get Nice Nails' 'Warnings'
 " * When you clip off the white tips of your nail, don't clip the nail into the shape you want, this will damage your nail badly. "
 'Just clip the majority of the ______ of the white bit off in one cl

In [2]:
import numpy as np
import pandas as pd

In [7]:
test = pd.DataFrame([['t', 's'], ['a', 'g']], columns=['t1', 't2'])

In [16]:
t1 = ['2', '3', '4']
t2 = ['3', '4', '5']
pd.DataFrame(np.array([t1, t2]).T)

,0,1
0,2,3
1,3,4
2,4,5
